In [2]:
import pandas as pd
import tensorflow 
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard 
from datetime import datetime
from sklearn.pipeline import Pipeline
from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder 
import pickle  
label_encoder_gender = LabelEncoder()
one_hot_encoder = OneHotEncoder(sparse_output = False)

In [3]:
churn = pd.read_csv('Churn_modelling.csv')
churn = churn.drop(columns = ['RowNumber', 'CustomerId', 'Surname'])
churn.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [4]:
cols = churn.select_dtypes(include = 'object').columns

churn[cols] = churn[cols].astype('category')



In [5]:
idsx = churn.select_dtypes(include = 'category').columns
for i in idsx:
    if len(churn[i].cat.categories) <= 2:
        churn[i] =  label_encoder_gender.fit_transform(churn[i])
    else:
        churn[i] = churn[i]


In [6]:
geo_df = one_hot_encoder.fit_transform(churn[['Geography']])

geo_df = pd.DataFrame(geo_df, columns = one_hot_encoder.get_feature_names_out(['Geography']))
churn = pd.concat([churn.drop(columns = 'Geography'), geo_df], axis = 1)

In [7]:
churn

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,771,1,39,5,0.00,2,1,0,96270.64,0,1.0,0.0,0.0
9996,516,1,35,10,57369.61,1,1,1,101699.77,0,1.0,0.0,0.0
9997,709,0,36,7,0.00,1,0,1,42085.58,1,1.0,0.0,0.0
9998,772,1,42,3,75075.31,2,1,0,92888.52,1,0.0,1.0,0.0


In [8]:
with open('label_encoder_gender.pk1', 'wb') as file:
    pickle.dump(label_encoder_gender, file)

with open('one_hot_encoder.pk1', 'wb') as file:
    pickle.dump(one_hot_encoder, file)

In [9]:
y = churn['Exited']
x = churn.drop(columns = 'Exited')


x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, )


scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

In [10]:
opt = tensorflow.keras.optimizers.Adam(learning_rate = 0.01)
loss = tensorflow.keras.losses.BinaryCrossentropy()

def create_model(neurons = 32, layers = 1):
    model = Sequential()
    model.add(Dense(neurons, activation = 'relu', input_shape = (x_train.shape[1],)))

    for _ in range(layers - 1):
        model.add(Dense(neurons, activation = 'relu'))

    model.add(Dense(1, activation = 'sigmoid'))
    model.compile(optimizer = opt, loss = loss, metrics = ['accuracy'])

    return model

In [11]:
model = KerasClassifier(layers = 1, neurons = 32, build_fn = create_model, verbose = 1)

In [12]:
param_grid = {
    'neurons': [16, 32, 64, 128],
    'layers': [1, 2, 3],
    'batch_size': [10, 20],
    'epochs': [50, 100]
}

In [13]:
grid = GridSearchCV(estimator = model, param_grid = param_grid, n_jobs = -1, cv = 3)
grid_result = grid.fit(x_train, y_train)

print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

c:\Users\tariq\Desktop\Neural Network Project\venv\Lib\site-packages\scikeras\wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


Epoch 1/100


800/800 [==============================] - 3s 2ms/step - loss: 0.4026 - accuracy: 0.8296
Epoch 2/100
800/800 [==============================] - 1s 2ms/step - loss: 0.3605 - accuracy: 0.8530
Epoch 3/100
800/800 [==============================] - 1s 2ms/step - loss: 0.3568 - accuracy: 0.8531
Epoch 4/100
800/800 [==============================] - 1s 2ms/step - loss: 0.3531 - accuracy: 0.8503
Epoch 5/100
800/800 [==============================] - 1s 2ms/step - loss: 0.3494 - accuracy: 0.8572
Epoch 6/100
800/800 [==============================] - 1s 2ms/step - loss: 0.3474 - accuracy: 0.8568
Epoch 7/100
800/800 [==============================] - 1s 2ms/step - loss: 0.3482 - accuracy: 0.8544
Epoch 8/100
800/800 [==============================] - 1s 2ms/step - loss: 0.3459 - accuracy: 0.8583
Epoch 9/100
800/800 [==============================] - 1s 2ms/step - loss: 0.3468 - accuracy: 0.8549
Epoch 10/100
800/800 [==============================] - 1s 1ms/step - loss: 0.3441 - accu

In [14]:
with open('scaler.pk1', 'wb') as file:
    pickle.dump(scaler, file)

In [ ]:
model = Sequential([
    Dense(16, activation = 'relu', input_shape = (x_train.shape[1],)),
    Dense(1, activation = 'sigmoid')         
])

In [16]:
opt = tensorflow.keras.optimizers.Adam(learning_rate = 0.01)
loss = tensorflow.keras.losses.BinaryCrossentropy()

In [17]:
model.compile(optimizer = opt, loss = loss, metrics = ['accuracy'])

In [18]:
log_dir = 'logs/fit/' + datetime.now().strftime("%m_%d_%Y_%S_%M_%H_")
tensorflow_callback = TensorBoard(log_dir = log_dir, histogram_freq = 1)
early_stopping_callback = EarlyStopping(monitor = 'val_loss', patience = 10, restore_best_weights = True)

In [23]:
history = model.fit(
    x_train, y_train, validation_data = (x_test, y_test), epochs = 100, batch_size = 10,
    callbacks = [tensorflow_callback, early_stopping_callback]
)

Epoch 1/100
800/800 [==============================] - 2s 1ms/step - loss: 0.3515 - accuracy: 0.8571 - val_loss: 0.3515 - val_accuracy: 0.8620
Epoch 2/100
800/800 [==============================] - 1s 1ms/step - loss: 0.3456 - accuracy: 0.8597 - val_loss: 0.3580 - val_accuracy: 0.8555
Epoch 3/100
800/800 [==============================] - 1s 2ms/step - loss: 0.3386 - accuracy: 0.8611 - val_loss: 0.3458 - val_accuracy: 0.8575
Epoch 4/100
800/800 [==============================] - 1s 1ms/step - loss: 0.3354 - accuracy: 0.8630 - val_loss: 0.3509 - val_accuracy: 0.8630
Epoch 5/100
800/800 [==============================] - 1s 1ms/step - loss: 0.3321 - accuracy: 0.8635 - val_loss: 0.3552 - val_accuracy: 0.8595
Epoch 6/100
800/800 [==============================] - 1s 1ms/step - loss: 0.3322 - accuracy: 0.8646 - val_loss: 0.3625 - val_accuracy: 0.8595
Epoch 7/100
800/800 [==============================] - 1s 1ms/step - loss: 0.3308 - accuracy: 0.8649 - val_loss: 0.3503 - val_accuracy: 0.8600

In [24]:
model.save('model.h5')

c:\Users\tariq\Desktop\Neural Network Project\venv\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
%load_ext tensorboard

In [35]:
%tensorboard --logdir logs/fit

Reusing TensorBoard on port 6006 (pid 44596), started 1 day, 9:08:53 ago. (Use '!kill 44596' to kill it.)